In [6]:
from gurobipy import *
nodes = ['1', '2', '3', '4', '5', '6', '7']
cost = {
  ('1', '2'): 12,
  ('1', '3'): 10,
  ('1', '7'): 12,
  ('2', '1'): 12,
  ('2', '3'): 8,
  ('2', '4'): 12,
  ('3', '1'): 10,
  ('3', '2'): 8,
  ('3', '4'): 11,
  ('3', '5'): 3,
  ('3', '7'): 9,
  ('4', '2'): 12,
  ('4', '3'): 11,
  ('4', '5'): 11,
  ('4', '6'): 10,
  ('5', '3'): 3,
  ('5', '4'): 11,
  ('5', '6'): 6,
  ('5', '7'): 7,
  ('6', '4'): 10,
  ('6', '5'): 6,
  ('6', '7'): 9,
  ('7', '1'): 12,
  ('7', '3'): 9,
  ('7', '5'): 7,
  ('7', '6'): 9,
  }
arcs, cost = multidict(cost)
# Create optimization model
m = Model('TSP')
# Create variables
x = {}
u = {}
for i,j in arcs:
    if (i,j) in cost:
        x[i,j] = m.addVar(obj=cost[i,j], vtype = 'B',
                          name='x_%s%s' % (i, j))
N = len(nodes)
for i in nodes:
    if i != nodes[0]:
        u[i] = m.addVar(obj=0, name='u_%s' % i)
m.update()
# Constraint for sum of incoming links to j
for j in nodes:
     m.addConstr(quicksum(x[i,j]
               for i in nodes if (i != j and ((i,j) in cost))) == 1,
                    'incom_%s' % (j))
# Constraint for sum of outgoing links from i
for i in nodes:
     m.addConstr(quicksum(x[i,j]
               for j in nodes if (i != j and ((i,j) in cost))) == 1,
                    'outgo_%s' % (i))
# Subtour elimination constraints
for i,j in arcs:
    if i != nodes[0] and j != nodes[0]:
       m.addConstr(u[i] - u[j] + N*x[i,j] <= N-1,
                'subtour_%s_%s' % (i, j))
# Compute optimal solution
m.optimize()
# Print solution
if m.status == GRB.Status.OPTIMAL:
    print('objective: %f' % m.ObjVal)
    solution = m.getAttr('x', x)
    for i,j in arcs:
        if (i,j) in cost:
            if solution[i,j] > 0:
                print('%s -> %s: %g' % (i, j, solution[i,j]))


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 34 rows, 32 columns and 112 nonzeros
Model fingerprint: 0x91bf073d
Variable types: 6 continuous, 26 integer (26 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 65.0000000
Presolve time: 0.00s
Presolved: 34 rows, 32 columns, 112 nonzeros
Variable types: 6 continuous, 26 integer (26 binary)

Root relaxation: objective 6.128571e+01, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   61.28571    0    6   65.00000   61.28571  5.71%     -    0s
H    0     0                      64.0000000   61.285